# http://tau-data.id/umi/

<center><img alt="" src="images/0_Cover.jpg"/></center> 

## <center><font color="blue">Modul 05: Pendahuluan Machine Learning 01</font></center>
<b><center>(C) Taufik Sutanto - 2019</center>
<center>tau-data Indonesia ~ https://tau-data.id ~ taufik@tau-data.id</center>

## <center><font color="blue">Pendahuluan Machine Learning 1: Unsupervised Learning & Regresi</font></center>
<img alt="" src="images/PDS_logo.jpg" />

## <font color="green">Hari ke-3 (Rabu, 29 Jan 2020)</font>

**Pendahuluan Data Science II**
* 09:00 – 11:00 Unsupervised Learning & Regresi
* 11:00 – 12:00	Latihan
* 13:00 – 15:00	Supervised Learning - Klasifikasi
* 15:00 – 16.00	Latihan Klasifikasi

Studi Kasus: **Human Resources Analytics**

## Outline:

* Pendahuluan Clustering
* Centroid-Based Clustering: k-Means, k-Means++, Minibatch k-means
* Evaluasi Centroid-based clusetering: MSE dan Silhouette Coefficient
* Optimal Clusters?
* Visualisasi dan Interpretasi Centroid Based Clustering
* Density Based Clustering dan Multivariate outlier Detection (DBSCAN)
* Pendahuluan Hierarchical Clustering

* Hierarchical Links

In [ ]:
!wget https://raw.githubusercontent.com/taufikedys/UMI/master/tau_unsup.py
!pip install umap-learn
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/Mall_Customers.csv

In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import time, umap, numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
import tau_unsup as tau, scipy.stats as stats
import scipy, itertools
from matplotlib.colors import ListedColormap
from sklearn import cluster, datasets
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn.metrics import silhouette_score as siluet
from sklearn.metrics.cluster import homogeneity_score as purity
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import confusion_matrix, classification_report
from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
plt.style.use('bmh'); sns.set();sns.set(style="ticks", color_codes=True)
random_state = 170

<p><img alt="" src="images/clustering_Quotes.png" style="width: 800px; height: 118px;" /></p>


# Clustering?

<p><img alt="" src="images/5_what_is_clustering.png" style="height:457px; width:864px" /></p>

## Definition

Clustering is as a process of finding group structures within data such that each instance within a group is similar to one another and dissimilar to instances in other groups [1]

[1]. Jain, A.K., Data clustering: 50 years beyond K-means. Pattern Recognition Letters, 2010. 31(8): p. 651-666.

## Applications

Clustering analysis applications can be divided into two broad categories, clustering for utility (e.g., data compression and indexing) and clustering for understanding data (e.g., finding latent structures or insights in the data) [2]. Methods developed in this subject (Data Mining) fall into the second category.

[2]. Pang-Ning, T., M. Steinbach, and V. Kumar, Introduction to data mining. Vol. 74. 2006, Boston, MA, USA: Addison-Wesley Longman Publishing Co., Inc.

## Realworld Clustering Applications

* Recommendation engines
* Market segmentation
* Social network analysis
* Search result grouping
* Medical imaging
* Image segmentation
* Anomaly detection

<p><img alt="" src="images/5_Clus-Clas.png" style="height:500px; width:766px" /></p>

<p><img alt="" src="images/5_types_of_clustering.png" style="height:500px; width:719px" /></p>

# Tantangan Clustering

* Computational Complexity
* Evaluation
* Interpretation
* Heavily depends on domain knowledge

<p><img alt="" src="images/iris_flower.png" /></p>

In [ ]:
# Kita akan menggunakan 2 data: Iris
data = load_iris()
type(data)

In [ ]:
print(data.DESCR)

In [ ]:
print(data.feature_names)
print(data.target_names)

In [ ]:
X_labels = data.feature_names
Y_labels = data.target_names
X = data.data
y = data.target
print(X.shape, len(y))

In [ ]:
print(y)

In [ ]:
print(X)

<h2>k-Means</h2>

<p><img alt="" src="images/5_Kmeans_animation.gif" style="height:300px; width:300px" /></p>


<p><img alt="" src="images/5_kmeans_Algorithm.png" style="height:229px; width:430px" /></p>

## How it works

* Numerically
* Visually: https://www.learndatasci.com/tutorials/k-means-clustering-algorithms-python-intro/ 

<ul>
	<li>
	<h3>Apakah pengaruh menggunakan centroid dan algoritma ini terhadap hasil cluster?</h3>
	</li>
	<li>
	<h3>k-Means tidak Robust terhadap outlier, Why? ==&gt; lalu apa yang sebaiknya dilakukan?</h3>
	</li>
</ul>

In [ ]:
# k-means: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

k = 3
km = cluster.KMeans(n_clusters=k, init='random', max_iter=300, tol=0.0001, n_jobs=-1, random_state = random_state)
km.fit(X)
'Done'

In [ ]:
# Hasil clusteringnya
C_km = km.predict(X)
C_km
# Apa beda label ini dengan klasifikasi ("labels")?

In [ ]:
plot = sns.countplot(C_km)

In [ ]:
X2D = umap.UMAP(n_neighbors=5, min_dist=0.3).fit_transform(X)
fig, ax = plt.subplots()
ax.scatter(X2D[:,0], X2D[:,1], c=C_km)
plt.show()

<h3 id="k-Means++">k-Means++</h3>

<p>Original <em>k-means</em> memulai algoritmanya dengan mengacak centroid awal dan k-means tidak &quot;robust&quot; terhadap centroid awal ini (apa artinya?).</p>

<p>&nbsp;</p>

<p><strong>k-Means akan menghasilkan hasil yang berbeda-beda jika di-run beberapa kali!....</strong></p>

<p>k-Means++ &quot;mengatasi&quot; hal ini:</p>

<p>inisialisasi centroid tidak random, tapi dengan menghitung probabilitas terbaik bagi centroid awal.</p>

<p>Keuntungan selain lebih robust, biasanya iterasi yang dibutuhkan jauh lebih sedikit ketimbang k-means++</p>

<p>Reference :&nbsp;<a href="http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf" target="_blank">http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf</a>&nbsp;</p>


In [ ]:
# k-means++ clustering http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
kmPP = cluster.KMeans(n_clusters=k, init='k-means++', max_iter=300, tol=0.0001, n_jobs=-1, random_state = random_state)
kmPP.fit(X)
C_kmpp = kmPP.predict(X)

plot = sns.countplot(C_kmpp)
C_kmpp[:10]

In [ ]:
fig, ax = plt.subplots()
plot = ax.scatter(X2D[:,0], X2D[:,1], c=C_kmpp)

<h3>Mini-Batch k-Means:</h3>

<p><img alt="" src="images/5_minibatch.JPG" style="height:350px; width:458px" /></p>

<p>&nbsp;</p>

<p><strong>Referensi</strong>: *Sculley, D. (2010, April). Web-scale k-means clustering. In&nbsp;<em>Proceedings of the 19th international conference on World wide web</em>&nbsp;(pp. 1177-1178). ACM.</p>

<p>* Google</p>


In [ ]:
# MiniBatch k-Means 
# http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html
# minibatch "tidak bisa parallel"!!!...
# parameter penting km = batch_size ... pada aplikasi sesungguhnya disarankan "minimal" 3xk
mbkm = cluster.MiniBatchKMeans(n_clusters=k, init='random', max_iter=300, tol=0.0001, random_state = random_state) 
mbkm.fit(X)

C_mbkm = mbkm.predict(X)
sns.countplot(C_mbkm)
C_mbkm[:10]

In [ ]:
fig, ax = plt.subplots()
plot = ax.scatter(X2D[:,0], X2D[:,1], c=C_mbkm)

In [ ]:
# MiniBatch k-Means++
mbkmPP = cluster.MiniBatchKMeans(n_clusters=k, init='k-means++', max_iter=300, tol=0.0001, random_state = random_state) 
mbkmPP.fit(X)
C_mbkmPP = mbkmPP.predict(X)
plot = sns.countplot(C_mbkmPP)
C_mbkmPP[:10]

In [ ]:
fig, ax = plt.subplots()
plot = ax.scatter(X2D[:,0], X2D[:,1], c=C_mbkmPP)


## k_means VS MiniBatch k-Means?

In [ ]:
tau.km_vs_mbkm()

## Evaluasi dan interpretasi k-Means

<h2 id="Silhouette-Coefficient">Silhouette Coefficient</h2>

<img alt="" src="images/silhouette.png" style="width: 800px; height: 391px;" />

* Apa makna intuitifnya?

In [ ]:
#Evaluasi : Internal . Contoh Silouette Coefficient ==> warning hanya cocok untuk k-means (centroid-based clustering)
Hasil_Clustering = [C_km, C_kmpp, C_mbkm, C_mbkmPP]
for res in Hasil_Clustering:
    print(siluet(X,res), end=', ')
# Bagaimana cara kerja dan interpretasinya?

<p><strong>Evaluasi Clustering: Internal VS External</strong></p>

<p><img alt="" src="images/5_Clustering_evaluation.jpg" style="height:400px; width:533px" /></p>


In [ ]:
# Bagaimana dengan evaluasi External?
# "C" adalah ground truth/golden standard
for res in Hasil_Clustering:
    print(purity(y,res), end=', ')

## Optimal Number of Clusters? - Elbow Method

In [ ]:
distorsions, k1, kN = [], 2, 10
for k in range(k1, kN):
    kmeans = cluster.KMeans(n_clusters=k).fit(X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(k1, kN), distorsions); plt.grid(True)
plt.title('Elbow curve')

## Optimal Number of Cluster based on Silhouette Score

In [ ]:
tau.sil_based_optimal_km()

# Interpretasi

In [ ]:
# Evaluasi sebenarnya tidak terlalu penting di Unsupervised learning.
# inilah yang membedakan "clustering" dan "clustering Analysis"
# yang lebih penting adalah interpretasi, tapi Bagaimana?
# contoh k-means
centroids = kmPP.cluster_centers_
centroids # perhatikan ada k centroids, karena ada k cluster
print(centroids[0].shape)
centroids

# Best/Optimal Clustering?

### Catatan Penting dalam mengevaluasi Clustering secara internal:

* Tidak ada clustering yang "benar"
* Yang terpenting adalah interpretability/Informasi yang didapatkan (non-trivial information)
* Internal metric tertentu hanya cocok untuk suatu algoritma tertentu juga, sehingga di Penelitian/Aplikasi di dunia professional jangan membandingkan 2 macam clustering dengan ukuran internal yang spesifik untuk metode clustering tertentu (misal Silhouette untuk k-Means).
* Kleinberg, J. M. (2003). An impossibility theorem for clustering. In Advances in neural information processing systems (pp. 463-470).
* Referensi 1: http://papers.nips.cc/paper/2340-an-impossibility-theorem-for-clustering.pdf
* Referensi 2: https://core.ac.uk/download/pdf/34638775.pdf

# DBSCAN

<p><img alt="" src="images/5_DBSCAN.gif" style="height:360px; width:690px" /></p>

<p>Karena algoritma (cara kerjanya) ini maka DBSCAN sering digunakan untuk (multivariate) outlier detection.</p>


In [ ]:
# DBSCAN http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
# tidak membutuhkan input parameter k!!!... sangat diperlukan untuk text Mining ... atau clustering data yang besar
dbscan = cluster.DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
dbscan.fit(X)

C_db = dbscan.labels_.astype(np.int)
plot = sns.countplot(C_db)
C_db[:10]
# apa makna cluster label -1?

In [ ]:
sum([1 for i in C_db if i==-1])

In [ ]:
fig, ax = plt.subplots()
plot = ax.scatter(X2D[:,0], X2D[:,1], c=C_db)

<p><img alt="" src="images/5_clustering_benchmarks.png" style="height:500px; width:962px" /></p>


<p><img alt="" src="images/5_Cluster_yg_mana.png" style="height:400px; width:694px" /></p>


<h2 id="Computational-Complexity-Challenge-of-Clustering">Computational Complexity Challenge of Clustering</h2>

<p><img alt="" src="images/clustering_efficiency.png" /></p>


# Analysis Regresi

<p><img alt="" src="images/RegressionModels.jpg" /></p>

## Outline Regresi:

* Pengantar ke Predictive Modelling
* Korelasi
* Regresi
* Evaluasi dan Interpretasi Model Regresi
* Regresi Logistik
* Evaluasi dan Interpretasi Model Klasifikasi

# Pendahuluan Model Regresi

* Digunakan saat variabel tak bebas (**Dependent variable** - Y) bertipe **numerik** (float/real) dan variabel bebasnya bisa numerik dan-atau kategorik

<p><img alt="" src="images/Supervised_Methods.png" /></p>


<p><img alt="" src="images/models_wrong_some_useful.png" /></p>

* Perfect/true-best model tidak ada, bahkan seringnya tidak diperlukan

# Beberapa contoh aplikasi regresi

1. **Predictive** Analytics: Memprediksi resiko, harga, penjualan, demand, dsb.

2. Operation Efficiency: Optimasi proses bisnis dengan melihat **hubungan antar variabel** dan mengambil policy berdasarkan hubungan tersebut.

3. Supporting Decisions: **Testing hypothesis**, misal terkait keuangan, operations dan customer purchases.

4. New **Insights**: Regresi dapat membantu menganalisa hubungan antar variabel dan sekaligus mem-filternya.

Sumber: https://www.newgenapps.com/blog/business-applications-uses-regression-analysis-advantages

# Regresi dan Korelasi

<p><img alt="" src="images/correlation_2.png" /></p>

# Regresi dan Korelasi

* Konsep dasar Persamaan/model Regressi adalah hubungan linear antara variabel bebas dan tak bebas.
* Sehingga kita akan awali bahasannya dengan Korelasi

<p><img alt="" src="images/correlation.png" /></p>


# Koefisien Korelasi Pearson

* Korelasi adalah suatu pengukuran untuk melihat **hubungan linier** antara dua variable numerik. Disimbolkan dengan **r**, untuk sampel, dan **$\rho$** untuk populasi.

* Langkah pertama dalam penentuan korelasi adalah dengan membuat **diagram pencar** (scatter plot) dari variabel terkait. Pada korelasi, kedua variable setara, dalam artian tidak ada yang menjadi variable bebas dan terikat. 

* Namun, untuk mempermudah pembahasan, pada diagram pencar, satu variable akan disimbolkan sebagai *x* (variable bebas) dan satu variable sebagai *y* (variable terikat/tak bebas). Jika titik – titik data terkonsentrasi di sekitar garis lurus, maka merupakan indikasi bahwa korelasi antara kedua variable tinggi. Makin terpencar data dari suatu garis lurus, menunjukkan makin rendah korelasi antara kedua variable tersebut.

# Nilai koefisien korelasi Pearson

* Nilai dari koefisien korelasi Pearson adalah dari -1 hingga +1.

<p><img alt="" src="images/korelasi_pearson_01.png" /></p>

<p><img alt="" src="images/linear-nonlinear-relationships.png" style="width: 688px; height: 266px;" /></p>

# Hati-hati

* Koefisien korelasi = 0 bukan berarti tidak ada hubungan antara kedua variable. Yang benar adalah: tidak ada hubungan LINIER, tapi bisa jadi ada hubungan dalam bentuk lain; misal: kuadratik, atau fungsi lain selain linier, seperti pada contoh di atas. 

# Memahami Korelasi dari perumusannya (Statistical Thinking)

* Korelasi sebenarnya adalah Covariance dibagi dengan masing-masing standar deviasinya.
* Apa maksud/maknanya?

<p><img alt="" src="images/korelasi_pearson_formulae.png" /></p>

# Contoh sederhana

<p><img alt="" src="img/data_korelasi_01.png" /></p>

In [ ]:
# ToyData Example
x = np.array([40, 45, 53, 60, 65, 71]) # Usia
y = np.array([126, 124, 135, 142, 139, 151]) # Tekanan Darah
np.corrcoef(x, y)
# Hasilnya adalah matriks korelasi

In [ ]:
# Plot untuk melihat datanya
plt.scatter(x, y)
plt.show()

In [ ]:
# cara lain dengan DataFrame
df = pd.DataFrame({'usia':x, 'Tekanan_Darah':y})
print(df.corr())
sns.heatmap(df.corr(),cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,annot=True, annot_kws={"size": 8}, square=True)
p = sns.pairplot(df)

# Interpretasi

* Nilai ~0.95 menunjukkan bahwa ada korelasi linier positif yang kuat antara usia dan tekanan darah. Ada kecenderungan bahwa usia tinggi berkaitan dengan tekanan darah yang kebih tinggi dibandingkan usia rendah.
* **WARNING**
* **Korelasi tidak sama (meng-implikasikan) dengan sebab akibat**. Perhatikan interpretasi di atas. Tidak dinyatakan bahwa jika usia tinggi maka tekanan darah rendah, hanya suatu tren atau kecenderungan. Mungkin saja usia dengan bertambahnya usia maka tekanan darah meningkat, tapi mungkin juga tekanan darah tinggi bukan karena usia, tapi faktor lain yang tidak teramati pada data.

* Contoh lain penelitian di Machine learning (kecantikan dan confidence/Panjang Jari dan IQ)

# Batasan nilai $R^2$ seperti ini? ... Really? Why? Why not?

<p><img alt="" src="images/Corr_interpretation.png" /></p>
[image Source: https://spencermath.weebly.com/home/interpreting-the-correlation-coefficient]

* Cases (social, medicine, etc)
* Objective, prediction vs insights.

# Regresi Linier Sederhana


<img alt="" src="images/Reg_sederhana_01.png"/>

<p><img alt="" src="images/reg_eq.png" style="width: 599px; height: 249px;" /></p>

<p><img alt="" src="images/reg_types.png" /></p>

[image source: https://www.slideshare.net/mahakvijay3/basics-of-regression-analysis]

<p><img alt="" src="images/Regression_FlowChart.jpg" /></p>

[image Source: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html]

# Korelasi ke Regresi

<p><img alt="" src="images/korelasi_2_regresi.png" /></p>

# Diskusi

* Mengapa Error diasumsikan berdistribusi normal?
* Error VS Residual?
* Di dunia nyata kita seringnya tidak mengetahui error. Loh kok? Maksudnya?
* Apa beda regresi dan interpolasi di Metode Numerik?

# Bahas sebentar pentingnya memahami "Loss Function"

* Persamaan/Model Linier adalah dasar terpenting di Statistika, Data Science, Machine Learning, dan Deep Learning (*).
* Banyak model di (*) sebenarnya adalah fungsi linier, bahkan di masalah klasifikasi.
* Yang membedakan adalah "pemodelan/optimasi masalah/Loss Functionnya"

-- Keterangan lebih lanjut ... 

# Heteroskedasticity

<p><img alt="" src="img/heterokedastisitas.png" /></p>


# Regresi memprediksi rata-rata $y_i$ untuk $x_i$, maksudnya?

<h1 id="Asumsi-Klasik">Asumsi Klasik</h1>

* https://tau-data.id/asumsi-statistika-benci-butuh/
<p><img alt="" src="images/asumsi_reg.png" style="width: 800px; height: 298px;" /></p>

<p><img alt="" src="images/asumsi_reg_2.png" /></p>


# Some warnings in Regression

<p><img alt="" src="images/Warnings_on_Regresi.png" /></p>

# Multikolinearitas: Mengapa dan seberapa "robust"?

## Kapan kita tidak perlu hawatir tentang hal ini?

# Pembahasan tentang Scaling

# Pembahasan tentang Interpolasi dan bukan Ekstrapolasi 

## Kalau mau ekstrapolasi bagaimana?

# Bagaimana jika error tidak berdistribusi "normal"?

# Evaluasi Error (Mean Squared Error)

<p><img alt="" src="images/mse_regresi.png" /></p>

* Hati-hati,... perhatikan rumusnya dengan baik .... ia tidak robust terhadap outlier
* $\hat{y} = \beta_0 + \beta_1 x_1 + ... + \beta_n x_n$
* MSE = total jarak/selisih antara prediksi dan nilai dari data (sesungguhnya)
* RMSE = $\sqrt{MSE}$   ... why? 
* Evaluasi penting ketika kita ingin melakukan prediksi

# Evaluasi $R^2$: Model VS Tidak Pakai Model?

<p><img alt="" src="images/R_Square.png" /></p>

# Adjusted R-Squared? Why?

<p><img alt="" src="images/Adjusted_R_Squared.png" /></p>

# Pengaruh Variabel Tak Bebas ke Model


<p><img alt="" src="images/sig_var_di_Regresi.png" /></p>

* $SSR = SST - SSE = \sum{(y_i-\bar{y})^2} - \sum{(y_i-\hat{y_i})^2}$  

# p-value (bergantung distribusi; contoh kasus distribusi normal)?

<p><img alt="" src="images/p-value.png" /></p>

* Probabilitas membuat kesalahan
* Dari rumus di cell sebelumnya, ada "masalah" dengan formulasi p-value.

# Menangani Data Kategorik di Regresi

1. Numeric Encoding
2. Dummy variable encoding
3. One-Hot Encoding

In [ ]:
# Contoh Data

dt = [['female', 'New York', 'low', 4], ['female', 'London', 'medium', 3], ['male', 'New Delhi', 'high', 2]]
col = ['Gender', 'City', 'Temperature', 'Rating']

In [ ]:
data = pd.DataFrame(dt,columns=col)
print(data)

# Numeric encoding

* Lebih cocok untuk ordinal (terutama jika kategorinya cukup banyak)

In [ ]:
data['City_encoded'] = LabelEncoder().fit_transform(data['City'])
data.head()
# Hati-hati tidak ada urutan encodingnya, cocok untuk variabel nominal

# Customized Ordinal Numeric Encoding

In [ ]:
mapping = {'low':0, 'medium':1, 'high':2}
data['Temperature_encoded'] = data['Temperature'].map(mapping)
data.head()

# One-hot encoding

* Preferable in Machine learning community
* Variabel nominal

In [ ]:
Oh = pd.get_dummies(data['City'], prefix='City')
Oh
# Perhatikan, sekarang setiap kategori di variabel kategorik menjadi sebuah variabel baru
# tapi bagaimana memasukan hasil encoding ini kembali ke datanya?

In [ ]:
dt = pd.concat([data, Oh], axis=1)
dt.head()

# Dummy Variable (Statisticians)

* less parameter
* konsepnya seperti "binary system"
* Statistician tidak suka jika jumlah parameter berlebih
* default encoding beberapa modul

In [ ]:
dum = pd.get_dummies(data['City'], prefix='City', drop_first=True)
dum.head()

In [ ]:
pd.concat([data, dum], axis=1)

# Regresi Non-Linier?

## Why?
## Kapan tidak disarankan menambah kompleksitas model?
## Regression for insights VS regression for prediction.

## Masih linear terhadap parameter

<p><img alt="" src="images/linearisasi_regresi.png" /></p>
[image source: https://sites.google.com/site/apphysics1online/appendices/2-data-analysis/graph-linearization ]

# Pentingnya "scaling" di Regresi (atau clustering) untuk mencari insight dari data

<p><img alt="" src="images/scaling.png" /></p>

<p><img alt="" src="images/standardize.png" /></p>
<p><img alt="" src="images/minmax.png" /></p>

[image source: https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e ]

# Floating Point tidak sama dengan Bilangan Real

## Komputer tidak bisa menyimpan dan mengolah bilangan real
# Tidak Percaya?
## Mari hitung deret berikut dengan komputer (Python)
# $\sum_{n=1}^{10^6}0.1$
## Seharusnya hasilnya adalah?

In [ ]:
# Here is the result according to Python
dSum = 0 
for i in range(10**6): # ini artinya for i dari 0 sampai (10^6)-1
    dSum = dSum + 0.1 # or "dSum += 0.1"
print(dSum)
# is the result correct? Why?

# Sistem Floating Point

<p><img alt="" src="images/6_float_3.png" /></p>

# Studi Kasus

* Automotive MPG (miles-per-Galon) Data Set : https://archive.ics.uci.edu/ml/datasets/Auto+MPG
* Paper awal: Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning. In Proceedings on the Tenth International Conference of Machine Learning, 236-243, University of Massachusetts, Amherst. Morgan Kaufmann. 

Attribute Information:

1. **mpg**: continuous ==> Dependent/target variable
2. cylinders: multi-valued discrete 
3. displacement: continuous 
4. horsepower: continuous 
5. weight: continuous 
6. acceleration: continuous 
7. model year: multi-valued discrete 
8. origin: multi-valued discrete 
9. car name: string (unique for each instance)

In [ ]:
# Contoh
# Load DataFile CSV
try:
    df = pd.read_csv('data/iklan.csv') # run locally
except:
    !wget https://raw.githubusercontent.com/taufikedys/tau-data/master/data/iklan.csv # "Google Colab"
    df = pd.read_csv('iklan.csv') 
df.head()

In [ ]:
p = sns.pairplot(df)

In [ ]:
standardize = preprocessing.StandardScaler()
minMax = preprocessing.MinMaxScaler()

In [ ]:
st = standardize.fit_transform(df[['Iklan', 'Laba']]) # Harus numerik dan Tidak Boleh ada Null/missing values
st # hasilnya Numpy Array

In [ ]:
# Jika ingin dimasukkan lagi ke Dataframe awal
st_df = pd.DataFrame(st, columns=['Iklan', 'Laba']) # Merubah Numpy Array menjadi DataFrame
df2 = pd.concat([df, st_df], axis=1) # caranya sama dengan saat kita mendiskusikan encoding variabel kategorik
df2.head()

In [ ]:
# Similary untuk minMax
mm = minMax.fit_transform(df[['Iklan', 'Laba']])
mm_df = pd.DataFrame(mm, columns=['iklan_mm', 'laba_mm']) # Merubah Numpy Array menjadi DataFrame
df3 = pd.concat([df, mm_df], axis=1) # caranya sama dengan saat kita mendiskusikan encoding variabel kategorik
df3.head()

# Contoh Regresi di Python (Data Pengeluaran Biaya Iklan)

In [ ]:
# Load DataFile CSV
try:
    df = pd.read_csv('data/iklan.csv') # run locally
except:
    !wget https://raw.githubusercontent.com/taufikedys/tau-data/master/data/iklan.csv # "Google Colab"
    df = pd.read_csv('iklan.csv') 
df.head()

In [ ]:
p = sns.pairplot(df, hue="Tipe")

In [ ]:
df.drop("No", axis=1, inplace=True)
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Fitting model Regresi Sederhana
lm = smf.ols("Laba ~ Iklan", data=df[['Laba','Iklan']]).fit()
lm.summary()

In [ ]:
# Mean Squared Error
lm.mse_total

In [ ]:
# Perintah-perintah ini hanya untuk memplot hasilnya
xmin, xmax = df.Iklan.min(), df.Iklan.max()
X = np.linspace(xmin, xmax, 100)
# params[0] is the intercept (beta0)
# params[1] is the slope (beta1)
Y = lm.params[0] + lm.params[1] * X
# Perhatikan ada 2 perintah plot
plt.plot(df.Iklan, df.Laba, "o")
plt.plot(X, Y, color="darkgreen")
#Selanjutnya hanya perintah labelling
plt.xlabel("Investasi Iklan")
plt.ylabel("Laba")
plt.show()

In [ ]:
# Menggunakan Module "SeaBorn" jauh lebih mudah ketimbang MatplotLib
p = sns.regplot(df.Iklan, df.Laba)

In [ ]:
# Kita bisa prediksi dengan cara seperti ini:
lm.predict({'Iklan': [21, 23, 25]})

In [ ]:
# Kenormalan residuals
res = lm.resid # residuals
fig = sm.qqplot(res, stats.t, fit=True, line='45')
plt.show()

In [ ]:
# Regresi Berganda dengan menambahkan variabel kategorik "Tipe"
res = ols(formula='Laba ~ Iklan + C(Tipe)', data=df).fit()
res.summary()
# Perhatikan bentuk modelnya, terutama pada data kategorik

In [ ]:
# Regresi Berganda non-linear (mencoba transformasi logaritmik)
res = ols(formula='Laba ~ np.log(Iklan) + C(Tipe)', data=df).fit()
res.summary()

# Train-Test untuk menghindari Overfit

* Konsepnya generalisasi/"inferensi" Sample ==> Populasi, logical thinkingnya di aplikasi dunia nyata kita tidak memiliki nilai "y" (variabel target) yang sesungguhnya.
* Catatan: pemisahan train-test menjaga proporsi kategori pada masalah klasifikasi (tidak murni random atau split)

<p><img alt="" src="img/train_test.png" /></p>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['Iklan', 'Tipe']], df['Laba'], test_size=0.4)

In [ ]:
X_train.head()

# Belum Dibahas:

* Model Regresi lain: Regression-tree, Support Vector Regression, Neural Network for Regression, dll.
* Regresi untuk data khusus, misal Ridge Regression jika jumlah variabel banyak
* General Linear Model dengan interaksi yang lebih umum/kompleks, dsb

# Selingan

* Pada penelitian sebaiknya menggunakan data public (open source) atau data private (misal data dari Rumah Sakit/Kampus)?

# <center><font color="blue"> End of Module 05

<hr />
<p><img alt="" src="images/meme_04_regression.png" /></p>